### Project 5: Relational Database
### Author: Chad Bell
### Date: 3-10-2023
### Description: Recreate the PREVAIL database

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import psycopg2

In [2]:
# Establish a connection with the postgres database
con = psycopg2.connect(
    host="localhost",
    database="postgres",
    user="postgres",
    password="example_password")
# Create a cursor object
cur = con.cursor()

In [16]:
# con.commit()
con.rollback()

In [17]:
# TODO: Rename tables with tbl and primarykeys with pk to fix id's

# Create loan types table
cur.execute("DROP TABLE IF EXISTS loan_types;")
cur.execute("""CREATE TABLE loan_types (
    id serial PRIMARY KEY NOT NULL,
    name TEXT,
    interest_rate NUMERIC,
    loan_term NUMERIC,
    pay_frequency NUMERIC,
    maturity_bonus NUMERIC,
    referral_rate NUMERIC,
    type_notes TEXT
    );"""
            )

# Create loans table
cur.execute("DROP TABLE IF EXISTS loans;")
cur.execute("""CREATE TABLE loans (
    id serial PRIMARY KEY NOT NULL,
    initial_start_date DATE,
    qualified BOOLEAN,
    ira_num TEXT,
    loan_notes TEXT,
    partner_paid DATE,
    promissory_note BYTEA,
    FOREIGN KEY (id) REFERENCES loan_types (id)
  );"""
            )
# Create loan instances table
cur.execute("DROP TABLE IF EXISTS loan_instances;")
cur.execute("""CREATE TABLE loan_instances (
    id serial PRIMARY KEY NOT NULL,
    amount NUMERIC,
    stage TEXT,
    app_submitted TIMESTAMP,
    funds_wired TIMESTAMP,
    start_date DATE,
    date_paid TIMESTAMP,
    loan_num NUMERIC,
    instance_notes TEXT,
    FOREIGN KEY (id) REFERENCES loans (id)
  );"""
            )

# Create clients table
cur.execute("DROP TABLE IF EXISTS clients;")
cur.execute("""CREATE TABLE clients (
    id serial PRIMARY KEY NOT NULL,
    first_name TEXT,
    last_name TEXT,
    email TEXT,
    phone TEXT,
    address TEXT,
    city TEXT,
    state TEXT,
    zip TEXT,
    client_notes TEXT
    );"""
            )
# Create referring partners table
cur.execute("DROP TABLE IF EXISTS referring_partners;")
cur.execute("""CREATE TABLE referring_partners (
    id serial PRIMARY KEY NOT NULL,
    name TEXT,
    email TEXT,
    referral_rate NUMERIC,
    partner_notes TEXT
  );""")

# Many-to-many relationship between clients and referring partners
cur.execute("DROP TABLE IF EXISTS client_referring_partner;")
cur.execute("""CREATE TABLE client_referring_partner (
    FOREIGN KEY (id) REFERENCES clients (id),
    FOREIGN KEY (id) REFERENCES referring_partners (id),
    PRIMARY KEY (client, referring_partner)
  );"""
            )

# Many-to-many relationship between loans and clients
cur.execute("DROP TABLE IF EXISTS loan_client;")
cur.execute("""CREATE TABLE loan_client (
    FOREIGN KEY (loan) REFERENCES loans (id),
    FOREIGN KEY (client) REFERENCES clients (id),
    PRIMARY KEY (loan, client)
  );"""
            )

UndefinedColumn: column "loan" referenced in foreign key constraint does not exist


In [6]:
cur.execute("SELECT * FROM information_schema.tables WHERE table_schema='public';")
cur.fetchall()

[]

In [8]:
# Functions to create:
# Reporting features
# 1. Create pay period report

# Basic functions
# 1. Create a new client
# 2. Create a new loan
# 3. Create a new loan instance
# 4. Create a new loan type
# 5. Create a new referring partner


# Requirements
"""You must create a database for your program to use including at least one table to store data into.
Your software must demonstrate the ability to insert, modify, delete, and retrieve (or query) data.
This module requires more than just creating the database and determining SQL commands. You must write software that builds the SQL commands, submits them, receives the results from the database, and uses the results in some way."""
"""Stretch: Add additional tables to your database and perform a join in your software between two of the tables.
"""
